In [1]:
import tensorflow as tf
from keras.utils import load_img
from keras.utils import img_to_array
from keras.applications.resnet import preprocess_input
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Flatten, Dropout, Input
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [15]:
features_df = pd.read_csv(r"C:\Users\rocci\OneDrive\Desktop\Università\DL\myCode\AutoencoderFeatureExtraction.csv")

In [3]:
features2 = pd.read_csv(r"C:\Users\rocci\OneDrive\Desktop\Università\DL\DiagClassifierTransferLearning-main\DiagClassifierTransferLearning-main\updatedDataset.csv")

In [19]:
features_df = features_df.drop(('Unnamed: 0'), axis = 1)

In [20]:
features_df = features_df.drop(('diag'), axis = 1)

In [21]:
features_df.head

<bound method NDFrame.head of             0          1          2         3         4          5         6  \
0    523.3593  447.37094  231.37323  361.1829  382.1728  320.71783  39.30802   
1    523.3593  447.37094  231.37323  361.1829  382.1728  320.71783  39.30802   
2    523.3593  447.37094  231.37323  361.1829  382.1728  320.71783  39.30802   
3    523.3593  447.37094  231.37323  361.1829  382.1728  320.71783  39.30802   
4    523.3593  447.37094  231.37323  361.1829  382.1728  320.71783  39.30802   
..        ...        ...        ...       ...       ...        ...       ...   
115  523.3593  447.37094  231.37323  361.1829  382.1728  320.71783  39.30802   
116  523.3593  447.37094  231.37323  361.1829  382.1728  320.71783  39.30802   
117  523.3593  447.37094  231.37323  361.1829  382.1728  320.71783  39.30802   
118  523.3593  447.37094  231.37323  361.1829  382.1728  320.71783  39.30802   
119  523.3593  447.37094  231.37323  361.1829  382.1728  320.71783  39.30802   

         

In [23]:
for column in features2.columns:
    if column == 'ID' or column == 'diag':
        print(column)
    else:
        features_df[column] = features2[column]
features_df['diag'] = features2['diag']
print(features_df.columns)

ID
diag
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       'mean_altitude', 'var_altitude', 'median_altitude', 'mean_azimuth',
       'var_azimuth', 'median_azimuth', 'duration', 'penLifts',
       'intersectionsCount', 'diag'],
      dtype='object', length=10394)


In [24]:
len(features_df.columns)

10394

In [26]:
# Load the CSV file
data = features_df

# Split the data into input features (X) and labels (y)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the input features (optional but recommended)
#X_train = (X_train - X_train.mean()) / X_train.std()
#X_test = (X_test - X_test.mean()) / X_test.std()

In [27]:
for i in range(len(y_train)):
    if y_train[i] == 'DYSGR' or y_train[i] == 'diag':
        y_train[i] = 1.0
    if y_train[i] == '0':
        y_train[i] = 0.0
y_train = np.asarray(y_train).astype('float32')
y_train

array([0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0.,
       0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1.,
       1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0.], dtype=float32)

In [28]:
for i in range(len(y_test)):
    if y_test[i] == 'DYSGR' or y_test[i] == 'diag':
        y_test[i] = 1.0
    if y_test[i] == '0':
        y_test[i] = 0.0
y_test = np.asarray(y_test).astype('float32')
y_test

array([0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
       1., 1., 0., 1., 0., 0., 1.], dtype=float32)

In [31]:
# Define the model
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=10393))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 512)               5321728   
                                                                 
 dense_6 (Dense)             (None, 256)               131328    
                                                                 
 dense_7 (Dense)             (None, 128)               32896     
                                                                 
 dense_8 (Dense)             (None, 64)                8256      
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,494,273
Trainable params: 5,494,273
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
3/3 [==============================] - 2s 115ms/step - loss: 673.2925 - accuracy: 0.5104
Epoch 2/100
3/3 [==============================] - 0s 106ms/step - loss: 416.0377 - accuracy: 0.4896
Epoch 3/100
3/3 [==============================] - 0s 109ms/step - loss: 380.9096 - accuracy: 0.4792
Epoch 4/100
3/3 [==============================] - 0s 113ms/step - loss: 99.2182 - accuracy: 0.5729
Epoch 5/100
3/3 [==============================] - 0s 117ms/step - loss: 254.0551 - accuracy: 0.5729
Epoch 6/100
3/3 [==============================] - 0s 132ms/step - loss: 283.1892 - accuracy: 0.6250
Epoch 7/100
3/3 [==============================] - 0s 126ms/step - loss: 581.8503 - accuracy: 0.5833
Epoch 8/100
3/3 [==============================] - 0s 118ms/step - loss: 186.3642 - accuracy: 0.5104
Epoch 9/100
3/3 [==============================] - 0s 112ms/step - loss: 289.0201 - accuracy: 0.4583
Epoch 10/100
3/3 [==============================] - 0s 112ms/step - loss: 80.5055 - accuracy

In [33]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

1/1 [==============================] - 0s 212ms/step - loss: 7.6652 - accuracy: 0.5417
Test loss: 7.6652
Test accuracy: 0.5417


In [34]:
features_df.to_csv(r'C:\Users\rocci\OneDrive\Desktop\Università\DL\myCode\features2.csv', index = False)